1. Cargar tuits

In [2]:
import pandas as pd
import random

In [3]:
sample_rate = 0.1
chunk_size = 10000  # Adjust the chunk size as needed
all_chunks = []

for i in range(1,8):
    chunks = pd.read_csv(f'data/tweets_batch_00{i}.tsv', sep='\t',
                         usecols=["tweet_id", "user_id", "full_text"],
                         engine='python',
                         on_bad_lines='skip',
                         skiprows=lambda i: i>0 and random.random() > sample_rate,
                         chunksize=chunk_size)
    all_chunks += chunks

df = pd.concat(all_chunks, ignore_index=True)

for f in ["user_id"]:
    df[f] = df[f].astype("Int64")

In [4]:
df

tweet_id              user_id  \
0       1249125075124654080   924699708790108160   
1       1249125075846135808             36018626   
2       1249125076152320001            118864905   
3       1249125077020532740            233033221   
4       1249125076663971840            219686511   
...                     ...                  ...   
616224  1251661739734769664            151167071   
616225  1251661740284145665   896189817209401344   
616226  1251661743618584577            219793349   
616227  1251661744554102785  1050700467867652096   
616228  1251661748219686913   782058499798818816   

                                                full_text  
0       Eje Metropolitano ha sido priorizado con las j...  
1       AMLO pide a Trump le venda 10 mil ventiladores...  
2       Pueblo de Granada agradece donación de pruebas...  
3       Desafíos y oportunidades por el COVID-19 https...  
4       Si ya consultaste que en tu hogar nadie ha sid...  
...                                                   ...  
616224  Reporte vespertino #coronavirus @msalnacion\n\...  
616225  Adriana, una doctora que trabajaba en una resi...  
616226  Salió del hospital después de haberse recupera...  
616227  @Kiarita9621 @maryvarelaGH Se ven afectados me...  
616228  En Monterrey han iniciado un ensayo clínico al...  

[616229 rows x 3 columns]

In [6]:
len(df)

616229

In [7]:
docs = df
docs = docs.full_text.dropna().values.tolist()

In [8]:
len(docs)

616212

In [9]:
docs[:2]

['Eje Metropolitano ha sido priorizado con las jornadas de desinfección en espacios públicos, calles y avenidas por registrar la mayoría de los casos de COVID-19. #QuédateEnCasa #11Abr https://t.co/UmNE7aVpcJ https://t.co/eViyEKHQuh',
 'AMLO pide a Trump le venda 10 mil ventiladores y equipo para enfrentar el covid-19!\n#NOTIVER #VERACRUZ #NACIONALES \nhttps://t.co/HzD8nwvIYX']

## Hashtags populares

In [10]:
import re

import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return str(only_ascii)

def extract_hashtags(s):
    return [
        remove_accents(ht.lower())
        for ht in re.findall(r"#(\w+)", s)
    ]

In [11]:
df.head()

tweet_id             user_id  \
0  1249125075124654080  924699708790108160   
1  1249125075846135808            36018626   
2  1249125076152320001           118864905   
3  1249125077020532740           233033221   
4  1249125076663971840           219686511   

                                           full_text  
0  Eje Metropolitano ha sido priorizado con las j...  
1  AMLO pide a Trump le venda 10 mil ventiladores...  
2  Pueblo de Granada agradece donación de pruebas...  
3  Desafíos y oportunidades por el COVID-19 https...  
4  Si ya consultaste que en tu hogar nadie ha sid...

In [12]:
df["full_text"] = df.full_text.astype(str)

In [13]:
df["hashtags"] = df.full_text.apply(extract_hashtags)

In [14]:
df.head()

tweet_id             user_id  \
0  1249125075124654080  924699708790108160   
1  1249125075846135808            36018626   
2  1249125076152320001           118864905   
3  1249125077020532740           233033221   
4  1249125076663971840           219686511   

                                           full_text  \
0  Eje Metropolitano ha sido priorizado con las j...   
1  AMLO pide a Trump le venda 10 mil ventiladores...   
2  Pueblo de Granada agradece donación de pruebas...   
3  Desafíos y oportunidades por el COVID-19 https...   
4  Si ya consultaste que en tu hogar nadie ha sid...   

                                   hashtags  
0              [b'quedateencasa', b'11abr']  
1  [b'notiver', b'veracruz', b'nacionales']  
2                [b'hermandadenvanguardia']  
3                                        []  
4                      [b'alertacovid19sv']

In [15]:
from collections import Counter

In [16]:
ht_counts = Counter()

In [17]:
for hts in df.hashtags.values:
    ht_counts.update(hts)

In [18]:
ht_counts.most_common(100)

[("b'covid19'", 187704),
 ("b'quedateencasa'", 29871),
 ("b'coronavirus'", 23025),
 ("b'covid'", 10828),
 ("b'cuba'", 9372),
 ("b'covid_19'", 5910),
 ("b'cubasalvavidas'", 4324),
 ("b'atencionmedicadecalidad'", 4273),
 ("b'venezuela'", 4155),
 ("b'estevirusloparamosunidos'", 3904),
 ("b'yomequedoencasa'", 3418),
 ("b'eeuu'", 3201),
 ("b'laprevencioneslaclave'", 3023),
 ("b'yoapoyolacuarentena'", 2828),
 ("b'ultimahora'", 2731),
 ("b'covid19mx'", 2661),
 ("b'venezuelabellaenrevolucion'", 2637),
 ("b'cuarentena'", 2490),
 ("b'17abr'", 2426),
 ("b'concienciaycompromiso'", 2282),
 ("b'mexico'", 2202),
 ("b'cubaporlasalud'", 2020),
 ("b'17anosbarrioadentro'", 1976),
 ("b'abrildeunioncivicomilitar'", 1880),
 ("b'abrildevictoriapopular'", 1801),
 ("b'pandemia'", 1759),
 ("b'urgente'", 1737),
 ("b'reporte'", 1731),
 ("b'sanadistancia'", 1706),
 ("b'15abr'", 1687),
 ("b'envideo'", 1683),
 ("b'covid2019'", 1647),
 ("b'18abr'", 1611),
 ("b'atencion'", 1601),
 ("b'covid19chile'", 1597),
 ("b'enviv

In [19]:
len(df)

616229

In [20]:
dfc = df[df.hashtags.apply(lambda hts: b"cuarentena" in hts)]

In [21]:
for t in dfc.sample(100).full_text.values:
    print(t)
    print("--------------------")

ValueError: a must be greater than 0 unless no samples are taken